In [1]:
import os
import re
import time
import urllib

from django.core.files import File
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException

driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')

In [2]:
driver.get('https://www.dabangapp.com/')



In [10]:
div_list = driver.find_element_by_xpath('/html/body/div[1]/div/nav/div[2]/div[2]')
print(div_list)
image_list = []

for i, url in enumerate(div_list):
    cls_name = url.get_attribute('class')
    cls_name = cls_name.split(' ')
    cls_name = cls_name[1]
    photo = driver.execute_script(
        f'return window.getComputedStyle(document.querySelector(".{cls_name}"),":after").getPropertyValue("background")')

    test_url = re.findall(r'"(.*?)"', photo)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/nav/div[2]/div[2]"}
  (Session info: chrome=81.0.4044.113)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.14.4 x86_64)


In [11]:
salesmonthlyChar = '400만원'

salesmonthlyInt = salesmonthlyChar.replace('만원', '')
salesmonthlyInt = int(salesmonthlyInt)

In [12]:
salesmonthlyInt

400

# Recently User Post List

In [31]:
postList = '11,12,13,14'

postList

print(type(postList))

postlist = postList.split(',')

<class 'str'>


In [33]:
user_post_list = []
for i in postlist:
    i = int(i)
    post = PostRoom.objects.get(pk=i)
    user_post_list.append(post)


NameError: name 'PostRoom' is not defined

In [ ]:
social_user = RecentlyPostList.objects.filter(user=request.user.pk)
social_user_list = []
for post in social_user:
    social_user_list.append(post)
    
user_count = len(social_user_list)

if user_count >=5:
    social_user_list[0].delete()
    social_user = RecentlyPostList.objects.filter(user=request.user.pk)
    social_user_list = []
    for post in social_user:
        social_user_list.append(post)

In [ ]:
def recently_post(request):
    user = request.user
    post_count = len(user.posts.all())
    if post_count >= 7:
        
    

In [ ]:
def recursivePost(user):
    cou